# A Meeting Analysis Use Case

Copyright 2025-2026, Denis Rothman

The notebook's goal is to demonstrate how to guide an AI through a multi-step analytical process, moving from a raw transcript to actionable insights, thereby training both the user and the AI.

This notebook is a step-by-step guide on how you can engineer a structured context . This guide will serve as the architectural blueprint for your code, explaining not just the *what* but the *why* at each stage.


-----

### Setup and Preliminaries

This section handles the basic configuration.

1.  **Cell 1: Install Libraries**

      * Install the necessary OpenAI library.

In [1]:
# Cell 1: Installation
!pip install tqdm==4.67.1 --upgrade
!pip install openai==2.14.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 2.12.0
    Uninstalling openai-2.12.0:
      Successfully uninstalled openai-2.12.0


2.  **Cell 2: Imports and API Key**

      * Import the library and securely prompt for the user's API key. This is better than hard-coding it.

In [2]:
# Cell 2: Imports and API Key Setup
# We will use the OpenAI library to interact with the LLM and Google Colab's
# secret manager to securely access your API key.

import os
from openai import OpenAI
from google.colab import userdata

# Load the API key from Colab secrets, set the env var, then init the client
try:
    api_key = userdata.get("API_KEY")
    if not api_key:
        raise userdata.SecretNotFoundError("API_KEY not found.")

    # Set environment variable for downstream tools/libraries
    os.environ["OPENAI_API_KEY"] = api_key

    # Create client (will read from OPENAI_API_KEY)
    client = OpenAI()
    print("OpenAI API key loaded and environment variable set successfully.")

except userdata.SecretNotFoundError:
    print('Secret "API_KEY" not found.')
    print('Please add your OpenAI API key to the Colab Secrets Manager.')
except Exception as e:
    print(f"An error occurred while loading the API key: {e}")


OpenAI API key loaded and environment variable set successfully.


3.  **Cell 3: The Raw Data (The "Crime Scene")**

      * Define the meeting transcript as a multi-line string. This is our primary data source.

In [3]:

# Cell 3: The Full Workshop Transcript
meeting_transcript = """
        Alex: Alright, let’s get started! I hope everyone grabbed some of those mini-donuts in the breakroom. I think I’ve had three already, so I’m officially on a sugar high.
        Jamie: They’re dangerous, Alex. Anyway, I’m ready when you are.
        Sam: Same here. My calendar is clear for the next two hours.
        Alex: Great. So, the goal of this workshop is to finalize our "Lumina Design System" core components. First up: the Primary Button component. Jamie, I saw your note about the hover states?
        Jamie: Yeah, the current spec uses a 15% lighten filter for the hover state. It looks fine on Chrome, but it’s rendering inconsistently on some legacy mobile browsers. It’s a mess to maintain.
        Sam: Legacy browsers? I thought we were phasing those out?
        Jamie: We are, but we still have that one government contract that requires IE11 support for another six months.
        Sam: Right, the "Directorate" account. Okay, we can't break that. What’s the fix?
        Jamie: I suggest we move away from filters and just define a specific HEX code for the hover state. It’s more manual work for me to set up, but it’s bulletproof across all browsers.
        Alex: That makes sense. Let’s do it. Jamie, can you map those HEX codes by Friday?
        Jamie: I can try. I’m a bit swamped with the login page bug, but if I can get some help with the documentation, I can get it done.
        Sam: I can actually have our intern, Leo, help with the documentation part. He’s looking for a project.
        Alex: Perfect. Let’s tentatively assign the documentation to Leo, overseen by Jamie. Now, second topic: The Icon Library. We’re currently using "FontAwesome," but the licensing cost is going up next month.
        Sam: I saw that invoice. It’s a 40% jump! I’m not sure the budget can handle that right now.
        Jamie: We could switch to an open-source library like "Lucide" or "Phosphor." They’re free, but it would mean a full audit of every icon in the current app to make sure they match.
        Alex: (Sighs) That’s a massive undertaking. We have over 200 icons.
        Sam: What if we do a hybrid? We keep FontAwesome for the complex dashboard icons for now, but use Phosphor for all new features.
        Alex: I think that's a recipe for a visual disaster, Sam. The styles won't match. I’d actually propose we dedicate a "Sprint Zero" next month just to migrate everything to Lucide. It’s a one-time pain for long-term savings.
        Sam: I like the sound of "long-term savings." If Alex says the design will be cleaner, I’m on board. Let’s plan for that migration in the Q3 kickoff.
        Alex: Excellent. I’ll draft the migration roadmap. Okay, that’s the big stuff. Who wants the last donut?
        Jamie: I think I’m good. I need to go look at those HEX codes before my next meeting.
        """


-----

### Layer 1: Establishing the scope (The 'What')

Here, we define the scope of the analysis. Each step's output will inform the next, creating a chain of context.

1.  **Cell 4: g2 - Set the Signal-to-Noise Ratio**

      * We'll start by cleaning the data. The prompt explicitly tells the AI to separate substantive content from conversational noise.

In [4]:
# Cell 4: g2 - Isolating Content from Noise
prompt_g2 = f"""
        Analyze the following meeting transcript. Your task is to isolate the substantive content from the conversational noise.
        - Substantive content includes: decisions made, project updates, problems raised, and strategic suggestions.
        - Noise includes: greetings, pleasantries, and off-topic remarks (like coffee).
        Return ONLY the substantive content.

        Transcript:
        ---
        {meeting_transcript}
        ---
        """

In [5]:
from openai import OpenAI

try:
    client = OpenAI()

    response_g2 = client.chat.completions.create(
        model="gpt-5.2",
        messages=[
            {"role": "user", "content": prompt_g2}
        ]
    )

    substantive_content = response_g2.choices[0].message.content
    print("--- SUBSTANTIVE CONTENT ---")
    print(substantive_content)

except Exception as e:
    print(f"An error occurred: {e}")

--- SUBSTANTIVE CONTENT ---
- Workshop goal: finalize the **Lumina Design System** core components.

- **Primary Button hover state**
  - Problem: current hover spec uses a **15% lighten filter** that renders inconsistently on **legacy mobile browsers** and is hard to maintain.
  - Constraint: must maintain **IE11 support for six more months** due to the **Directorate (government) contract**.
  - Decision: **replace filter-based hover styling with explicit HEX hover colors** for cross-browser consistency.
  - Action items:
    - **Jamie** to map hover-state **HEX codes by Friday** (pending workload).
    - **Leo (intern)** assigned to help with **documentation**, overseen by **Jamie**.

- **Icon Library**
  - Update/Problem: **FontAwesome licensing cost increases 40% next month**, creating budget pressure.
  - Options discussed:
    - Switch to open-source (**Lucide** or **Phosphor**) but requires auditing/matching **200+ icons**.
    - Hybrid approach (keep FontAwesome for some, use P

2.  **Cell 5: g3 - Define the Scope of Time (Simulated RAG)**

      * We'll simulate a RAG context by providing a "previous" summary and asking for what's new. This teaches the user the importance of historical context.

In [6]:
# Cell 5: g3 - Identifying NEW Information (Simulated RAG)
previous_summary = "In our last meeting, we finalized the goals for Project Phoenix and assigned backend work to Tom and front-end to Maria."

prompt_g3 = f"""
Context: The summary of our last meeting was: "{previous_summary}"

Task: Analyze the following substantive content from our new meeting. Identify and summarize ONLY the new developments, problems, or decisions that have occurred since the last meeting.

New Meeting Content:
---
{substantive_content}
---
"""
from openai import OpenAI
from google.colab import userdata


# Your chat completion request
try:
    response_g3 = client.chat.completions.create(
        model="gpt-5.2",
        messages=[{"role": "user", "content": prompt_g3}]
    )
    new_developments = response_g3.choices[0].message.content
    print("--- NEW DEVELOPMENTS SINCE LAST MEETING ---")
    print(new_developments)
except Exception as e:
    print(f"An error occurred: {e}")


--- NEW DEVELOPMENTS SINCE LAST MEETING ---
- **Scope/Focus shift:** The workshop’s goal is now to finalize the **Lumina Design System** core components (new initiative not mentioned last time).

- **Primary Button hover state (new problem + decision):**
  - **Problem identified:** A **15% lighten filter** in the hover spec renders inconsistently on **legacy mobile browsers** and is difficult to maintain.
  - **Constraint noted:** Must keep **IE11 support for 6 more months** due to a **Directorate (government) contract**.
  - **Decision made:** Replace the filter-based hover styling with **explicit HEX hover colors** to ensure cross-browser consistency.
  - **New action items:**  
    - **Jamie** to map hover-state **HEX codes by Friday** (pending workload).  
    - **Leo (intern)** to assist with **documentation**, overseen by **Jamie**.

- **Icon Library (new problem + decision):**
  - **Problem identified:** **FontAwesome licensing costs** are increasing **40% next month**, creating

-----

### Layer 2: Conducting the Investigation (The 'How')

Now we move from identifying facts to generating insights, the core of the semantic context interpreation journey.

1.  **Cell 6: g4 - Identify the Key Threads**

      * This is a crucial step. The prompt asks the AI to read between the lines.

In [7]:
# Cell 6: g4 - Uncovering Implicit Threads
prompt_g4 = f"""
Task: Analyze the following meeting content for implicit social dynamics and unstated feelings. Go beyond the literal words.
- Did anyone seem hesitant or reluctant despite agreeing to something?
- Were there any underlying disagreements or tensions?
- What was the overall mood?

Meeting Content:
---
{substantive_content}
---
"""
try:
    response_g4 = client.chat.completions.create(
        model="gpt-5.2",
        messages=[{"role": "user", "content": prompt_g4}]
    )
    implicit_threads = response_g4.choices[0].message.content
    print("--- IMPLICIT THREADS AND DYNAMICS ---")
    print(implicit_threads)
except Exception as e:
    print(f"An error occurred: {e}")

--- IMPLICIT THREADS AND DYNAMICS ---
### Hesitation / reluctance (despite agreement)

- **Jamie’s Friday deadline reads as a soft commitment.** The parenthetical “*(pending workload)*” is a classic hedge: Jamie agreed to the action, but is signaling capacity risk and protecting themselves from being held to a hard date. That suggests reluctance to own a tight turnaround or frustration with current load—without openly pushing back.
- **Assigning Leo (intern) “to help with documentation, overseen by Jamie” subtly increases Jamie’s burden.** Even if framed as help, “overseen by Jamie” implies mentorship/review overhead. Jamie may not have objected in the moment, but the structure hints at a dynamic where work is delegated downward while accountability remains with Jamie—often accepted outwardly, felt as extra weight privately.
- **The IE11 constraint likely produced unspoken resignation.** “Must maintain IE11 support… due to the Directorate contract” is a non-negotiable external force. T

2.  **Cell 7: g5 - Perform "Intellectual Combinations"**

      * Here, we prompt the AI to be creative and solve a problem by synthesizing different ideas from the meeting.

In [8]:
# Cell 7: g5 - Generating a Novel Solution
prompt_g5 = f"""
Context: In the meeting, Maria suggested a 'soft launch' to avoid server strain, and also mentioned her team has 'extra bandwidth'.
Tom is facing a 3-day delay on the backend.

Task: Propose a novel, actionable idea that uses Maria's team's extra bandwidth to help mitigate Tom's 3-day delay. Combine these two separate pieces of information into a single solution.
"""
try:
    response_g5 = client.chat.completions.create(
        model="gpt-5.2",
        messages=[{"role": "user", "content": prompt_g5}]
    )
    novel_solution = response_g5.choices[0].message.content
    print("--- NOVEL SOLUTION PROPOSED BY AI ---")
    print(novel_solution)
except Exception as e:
    print(f"An error occurred: {e}")

--- NOVEL SOLUTION PROPOSED BY AI ---
Have Maria’s team use their extra bandwidth to build a **“soft-launch backend shim”** that reduces the amount of backend work needed on day one, letting you launch a limited version while Tom finishes the full backend over the next 3 days.

**Actionable plan**
1. **Define the soft-launch scope (today, 30–60 mins):** Agree on a small set of read-only or low-risk flows (e.g., browse, view details, limited onboarding) and cap traffic (invite-only, % rollout, or feature-flagged).
2. **Maria’s team implements a temporary adapter layer (today–tomorrow):**
   - Stand up a lightweight **API gateway / BFF (backend-for-frontend)** or edge function that:
     - Serves **stubbed responses** for endpoints Tom hasn’t finished (using static fixtures or a small temporary datastore).
     - Routes completed endpoints to Tom’s existing services.
     - Adds **aggressive caching** and request throttling to reduce server strain during the soft launch.
3. **Front-end +

-----

### Layer 3: Determining the Action (The 'What Next')

Finally, we turn the analysis into concrete, forward-looking artifacts.

1.  **Cell 8: g6 - Define the Output Format (Final Summary)**

      * We compile all the key information into a structured, final summary, showing the importance of clear outputs.

In [9]:
# Cell 8: g6 - Creating the Final, Structured Summary
prompt_g6 = f"""
Task: Create a final, concise summary of the meeting in a markdown table.
Use the following information to construct the table.

- New Developments: {new_developments}

The table should have three columns: "Topic", "Decision/Outcome", and "Owner".
"""
try:
    response_g6 = client.chat.completions.create(
        model="gpt-5.2",
        messages=[{"role": "user", "content": prompt_g6}]
    )
    final_summary_table = response_g6.choices[0].message.content
    print("--- FINAL MEETING SUMMARY TABLE ---")
    print(final_summary_table)
except Exception as e:
    print(f"An error occurred: {e}")

--- FINAL MEETING SUMMARY TABLE ---
| Topic | Decision/Outcome | Owner |
|---|---|---|
| Workshop scope/focus | Shifted goal to finalize **Lumina Design System** core components (new initiative). | Team |
| Primary Button hover state | Replaced **15% lighten filter** hover styling with **explicit HEX hover colors** for cross-browser consistency (must maintain **IE11 support for 6 more months** due to Directorate contract). | Jamie (HEX mapping); Leo (documentation support under Jamie) |
| Icon Library | Run **“Sprint Zero” next month** to migrate all icons from **FontAwesome to Lucide** (open-source); **hybrid approach rejected**. Migration included in **Q3 kickoff**. | Alex (migration roadmap) |


2.  **Cell 9: g7 - Generate the Subsequent Task**

      * The last step is to use the analysis to perform a real-world action, closing the loop from insight to action.

In [10]:
# Cell 9: g7 - Drafting the Follow-Up Action
prompt_g7 = f"""
Task: Based on the following summary table, draft a polite and professional follow-up email to the team (Sarah, Tom, Maria).
The email should clearly state the decisions made and the action items for each person.

Summary Table:
---
{final_summary_table}
---
"""
try:
    response_g7 = client.chat.completions.create(
        model="gpt-5.2",
        messages=[{"role": "user", "content": prompt_g7}]
    )
    follow_up_email = response_g7.choices[0].message.content
    print("--- DRAFT FOLLOW-UP EMAIL ---")
    print(follow_up_email)
except Exception as e:
    print(f"An error occurred: {e}")

--- DRAFT FOLLOW-UP EMAIL ---
Subject: Follow-up: Workshop decisions + action items (Lumina Design System)

Hi Sarah, Tom, and Maria,  

Thanks again for the discussion. I’m writing to recap the key decisions and confirm the action items/owners.

**Decisions confirmed**
- **Workshop scope/focus:** We’ve shifted the workshop goal to finalize the **Lumina Design System** *core components* (new initiative).
- **Primary Button hover state:** We will replace the **15% lighten filter** hover styling with **explicit HEX hover colors** to ensure cross-browser consistency. We also need to **maintain IE11 support for the next 6 months** due to the Directorate contract.
- **Icon Library:** We will run a **“Sprint Zero” next month** to migrate all icons from **FontAwesome to Lucide** (open-source). The **hybrid approach is rejected**, and the migration will be included in the **Q3 kickoff**.

**Action items**
- **Sarah / Tom / Maria (Team):** Align on the updated workshop objective and ensure upco

This notebook structure doesn't just "get a summary." It takes the user on a journey, showing them *how* to think with the AI as a partner. It perfectly translates the abstract "Scope, Investigation, Action" framework into a repeatable, educational, and powerful process.